In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os, random, joblib, statistics
import sklearn as sk
import optuna
import xgboost as xgb
import plotly
import json
import math
import matplotlib.pyplot as plt
from optuna.visualization import plot_optimization_history, plot_param_importances
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import svm, datasets
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.model_selection import GroupKFold
from typing import List, Optional, Tuple
import functions

random.seed(1213870)

/home/ucloud/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### LOADING IN ALL DATASETS AND PREPARING THEM

In [4]:
# data for model training 
path1 = ('../data/Preprop_data_w_folds')

data_MatchingGame = pd.read_csv(os.path.join(path1,'MatchingGame_w_folds.csv'), delimiter=';')
data_Questions_fam = pd.read_csv(os.path.join(path1, 'Questions_fam_w_folds.csv'), delimiter=';')
data_Questions_unfam = pd.read_csv(os.path.join(path1, 'Questions_unfam_w_folds.csv'), delimiter=';')



In [5]:
# data for model testing
path2 = ('../data/test_sets')

MG_MG_20 = pd.read_csv(os.path.join(path2,'MG/test_MatchingGame_20.csv'), delimiter=';')
MG_Qfam_20 = pd.read_csv(os.path.join(path2,'MG/test_Questions_fam_20.csv'), delimiter=';')
MG_Qfam_80 = pd.read_csv(os.path.join(path2,'MG/test_Questions_fam_80.csv'), delimiter=';')
Qfam_MG_20 = pd.read_csv(os.path.join(path2,'Qfam/test_MatchingGame_20.csv'), delimiter=';')
Qfam_MG_80 = pd.read_csv(os.path.join(path2,'Qfam/test_MatchingGame_80.csv'), delimiter=';')
Qfam_Qfam_20 = pd.read_csv(os.path.join(path2,'Qfam/test_Questions_fam_20.csv'), delimiter=';')
Qfam_Qunfam_20 = pd.read_csv(os.path.join(path2,'Qfam/test_Questions_unfam_20.csv'), delimiter=';')
Qfam_Qunfam_80 = pd.read_csv(os.path.join(path2,'Qfam/test_Questions_unfam_80.csv'), delimiter=';')
Qunfam_Qfam_20 = pd.read_csv(os.path.join(path2,'Qunfam/test_Questions_fam_20.csv'), delimiter=';')
Qunfam_Qfam_80 = pd.read_csv(os.path.join(path2,'Qunfam/test_Questions_fam_80.csv'), delimiter=';')
Qunfam_Qunfam_20 = pd.read_csv(os.path.join(path2,'Qunfam/test_Questions_unfam_20.csv'), delimiter=';')


In [6]:
# loading the feature lists
path3 = ('../data/feature_lists')

MG_features = pd.read_csv(os.path.join(path3,'MatchingGame_featurelist.csv'), delimiter=';')
Qfam_features = pd.read_csv(os.path.join(path3, 'Questions_fam_featurelist.csv'), delimiter=';')
Qunfam_features = pd.read_csv(os.path.join(path3, 'Questions_unfam_featurelist.csv'), delimiter=';')

In [7]:
# cleaning feature lists
functions.clean_features(MG_features, feature_list=True)
functions.clean_features(Qfam_features, feature_list=True)
functions.clean_features(Qunfam_features, feature_list=True)

# cleaning model training data
functions.clean_features(data_MatchingGame, feature_list = False)
functions.clean_features(data_Questions_fam, feature_list = False)
functions.clean_features(data_Questions_unfam, feature_list = False)

# cleaning test data
functions.clean_features(MG_MG_20, feature_list = False)
functions.clean_features(MG_Qfam_20, feature_list = False)
functions.clean_features(MG_Qfam_80, feature_list = False)
functions.clean_features(Qfam_MG_20, feature_list = False)
functions.clean_features(Qfam_MG_80, feature_list = False)
functions.clean_features(Qfam_Qfam_20, feature_list = False)
functions.clean_features(Qfam_Qunfam_20, feature_list = False)
functions.clean_features(Qfam_Qunfam_80, feature_list = False)
functions.clean_features(Qunfam_Qfam_20, feature_list = False)
functions.clean_features(Qunfam_Qfam_80, feature_list = False)
functions.clean_features(Qunfam_Qunfam_20, feature_list = False)


,median_F0final_sma,iqr_F0final_sma,median_voicingFinalUnclipped_sma,iqr_voicingFinalUnclipped_sma,median_jitterLocal_sma,iqr_jitterLocal_sma,median_jitterDDP_sma,iqr_jitterDDP_sma,median_shimmerLocal_sma,iqr_shimmerLocal_sma,...,iqr_mfcc_sma_de_11_,median_mfcc_sma_de_12_,iqr_mfcc_sma_de_12_,median_mfcc_sma_de_13_,iqr_mfcc_sma_de_13_,median_mfcc_sma_de_14_,iqr_mfcc_sma_de_14_,utterance_length,Diagnosis,ID
0,0.303231,0.108704,0.502144,0.835377,0.026343,0.123811,0.020299,0.105973,0.160442,0.312577,...,0.371626,0.518333,0.302256,0.396466,0.368206,0.463973,0.384397,0.098330,0,9611
1,0.260783,0.101097,0.653211,0.383666,0.021077,0.022190,0.020098,0.022383,0.103584,0.140880,...,0.282255,0.497142,0.335026,0.435754,0.321891,0.491925,0.404899,0.336207,0,9611
2,0.320001,0.062155,0.677215,0.884880,0.037941,0.090081,0.029462,0.138858,0.091119,0.369198,...,0.521192,0.506412,0.487988,0.422888,0.181943,0.509181,0.544741,0.071390,0,9611
3,0.259548,0.033698,0.767221,0.182358,0.017083,0.013999,0.016759,0.015936,0.085533,0.065614,...,0.186120,0.527404,0.289932,0.465167,0.221046,0.502223,0.226692,0.164781,0,9611
4,0.229530,0.015952,0.764407,0.208216,0.016090,0.016374,0.015124,0.017554,0.073327,0.063334,...,0.243267,0.555483,0.273139,0.457529,0.211052,0.465493,0.357992,0.090338,0,9611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3770,0.378669,0.131196,0.593775,0.254098,0.030057,0.025177,0.032243,0.023951,0.104500,0.085221,...,0.492774,0.565756,0.425799,0.447915,0.319424,0.499205,0.345846,0.119612,1,9795
3771,0.281573,0.094892,0.543637,0.391300,0.029904,0.049573,0.034030,0.045259,0.105900,0.161729,...,0.373334,0.539068,0.254839,0.465514,0.292250,0.468034,0.468639,0.226473,1,9795
3772,0.268596,0.072960,0.409935,0.391503,0.060665,0.057163,0.073796,0.049728,0.156550,0.170607,...,0.513214,0.514956,0.352729,0.363946,0.326063,0.456537,0.390571,0.130119,1,9795
3773,0.308245,0.393642,0.282741,0.637794,0.104317,0.089215,0.099860,0.088509,0.368260,0.373852,...,0.277646,0.611109,0.424392,0.478582,0.339883,0.438622,0.297590,0.037626,1,9795


# XGB

In [7]:
# HYPERPARAMETERTUNING
# NO NEED TO RUN ANYMORE: VALUES ARE SAVED AND CAN BE RUN INTO WD IN CHUNK UNDERNEATH
bestparams_MG_xgb = functions.parametertuner(model_type = 'xgb', train = data_MatchingGame, feature_lists = MG_features, train_name = 'MatchingGame', trials = 100, jobs = 32)

[I 2024-11-28 22:48:44,555] A new study created in memory with name: no-name-c3125f57-4d5c-4b90-9c97-20f9402afda1


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
22

2
2
2
2
2
2
2
2
2
2
2
3
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
3
3
3
3
4
4
3
3
3
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
5
4
5
4
5
4
4
4
5
44

5
4
5
4
5
5
4
5
5
5
5
4
5
5
5


[I 2024-11-28 22:49:56,207] Trial 24 finished with value: 0.5742195611964631 and parameters: {'learning_rate': 0.2862483422238572, 'reg_lambda': 1.0299171157361712e-06, 'max_depth': 6, 'min_child_weight': 33, 'gamma': 4.586521189289893, 'reg_alpha': 0.22230379882059137}. Best is trial 24 with value: 0.5742195611964631.


5
1
5
4


[I 2024-11-28 22:49:57,716] Trial 29 finished with value: 0.5720082937981986 and parameters: {'learning_rate': 0.24610661904288897, 'reg_lambda': 3.3333763971595603e-07, 'max_depth': 3, 'min_child_weight': 97, 'gamma': 4.9204492812099225, 'reg_alpha': 5.842927974562752e-06}. Best is trial 24 with value: 0.5742195611964631.


...

[I 2024-11-28 22:53:22,057] Trial 94 finished with value: 0.5734623685532766 and parameters: {'learning_rate': 0.0701930571225561, 'reg_lambda': 1.4224381390026172e-06, 'max_depth': 9, 'min_child_weight': 71, 'gamma': 2.2776981878536926, 'reg_alpha': 3.3693047653154586e-07}. Best is trial 73 with value: 0.5780589727033894.


4
4
4
5
5
5


[I 2024-11-28 22:53:25,197] Trial 96 finished with value: 0.5730185610020846 and parameters: {'learning_rate': 0.1834675713645324, 'reg_lambda': 1.0665675334149358e-06, 'max_depth': 9, 'min_child_weight': 76, 'gamma': 1.2692089323587816, 'reg_alpha': 0.015541467537250298}. Best is trial 73 with value: 0.5780589727033894.
[I 2024-11-28 22:53:25,672] Trial 97 finished with value: 0.5713483506562372 and parameters: {'learning_rate': 0.17429407448264353, 'reg_lambda': 0.0006201113719579643, 'max_depth': 9, 'min_child_weight': 76, 'gamma': 2.0388774702383525, 'reg_alpha': 0.021721138543814265}. Best is trial 73 with value: 0.5780589727033894.


5


[I 2024-11-28 22:53:26,544] Trial 99 finished with value: 0.5691172269039463 and parameters: {'learning_rate': 0.17075311879948946, 'reg_lambda': 0.0005176751270490144, 'max_depth': 9, 'min_child_weight': 80, 'gamma': 2.1333537469599677, 'reg_alpha': 3.0575123120595613e-07}. Best is trial 73 with value: 0.5780589727033894.
[I 2024-11-28 22:53:27,096] Trial 98 finished with value: 0.565668641725283 and parameters: {'learning_rate': 0.1699639041418185, 'reg_lambda': 0.0004906700617064091, 'max_depth': 9, 'min_child_weight': 47, 'gamma': 1.2976598311843104, 'reg_alpha': 0.01128762845231455}. Best is trial 73 with value: 0.5780589727033894.


In [8]:
# loading in best params
#bestparams_MG_xgb = functions.extract_bestparams('MatchingGame', model_type = 'xgb')

In [9]:
# training and testing MatchingGame model: OBS giving one prediction per participant

MG_training_xgb = functions.modeltraining(data_MatchingGame,
             MG_features,
             bestparams_MG_xgb,
             'xgb',
             'MatchingGame',
             save = True)

MG_MG_20_test_xgb = functions.modeltesting2(feature_lists = MG_features,
             test = MG_MG_20,
             model_type = 'xgb',
             save = True,
             test_name = "MG_MG_20",
             train_name = 'MatchingGame')

MG_Qfam_80_test_xgb = functions.modeltesting2(MG_features,
             MG_Qfam_80,
             'xgb',
             save = True,
             test_name = "MG_Qfam_80",
             train_name = 'MatchingGame')

MG_Qfam_20_test_xgb = functions.modeltesting2(MG_features,
             MG_Qfam_20,
             'xgb',
             save = True,
             test_name = "MG_Qfam_20",
             train_name = 'MatchingGame')

# SVM

In [10]:
# SVM PARAMETERTUNING
bestparams_MG_svm = functions.parametertuner(model_type = 'svm', train = data_MatchingGame, feature_lists = MG_features, train_name = 'MatchingGame', trials = 100, jobs = 32)

[I 2024-11-28 22:53:34,542] A new study created in memory with name: no-name-bef54671-e2b6-4e91-88ba-31fc4fb8c997


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
3
2
2
2
2
2
2
2
2
2
2
2
4
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
5
3


[I 2024-11-28 22:58:05,455] Trial 10 finished with value: 0.564105649430214 and parameters: {'C': 65.66413028694177, 'gamma': 0.3602224176687265}. Best is trial 10 with value: 0.564105649430214.


1
3
3
3
3
3
3
3
3
3
3
3
3
3
4
3
3
3
3
3
3
3
3
3
3
4
3
3
3
3
3
3
3
2
4
4
4
4
4
4
5
4
4
4
5
4
4


[I 2024-11-28 23:03:20,119] Trial 1 finished with value: 0.5566658118651104 and parameters: {'C': 36.02447881400141, 'gamma': 4.25831699240759}. Best is trial 10 with value: 0.564105649430214.


...

[I 2024-11-28 23:33:47,445] Trial 84 finished with value: 0.48390214043659013 and parameters: {'C': 58.18423710802499, 'gamma': 15.609585430880406}. Best is trial 66 with value: 0.5701049939691195.


5
5
5


[I 2024-11-28 23:34:03,838] Trial 86 finished with value: 0.4834410382098738 and parameters: {'C': 39.2501561428522, 'gamma': 15.798661286133049}. Best is trial 66 with value: 0.5701049939691195.


5
5


[I 2024-11-28 23:34:16,787] Trial 81 finished with value: 0.4822469328903941 and parameters: {'C': 53.987833248692816, 'gamma': 16.12780759135785}. Best is trial 66 with value: 0.5701049939691195.
[I 2024-11-28 23:34:30,214] Trial 82 finished with value: 0.4794102885196379 and parameters: {'C': 56.41315323227902, 'gamma': 16.56011820086546}. Best is trial 66 with value: 0.5701049939691195.
[I 2024-11-28 23:34:33,660] Trial 93 finished with value: 0.5414888176736549 and parameters: {'C': 39.05590281602551, 'gamma': 7.0929992857863216}. Best is trial 66 with value: 0.5701049939691195.
[I 2024-11-28 23:34:33,663] Trial 92 finished with value: 0.5434375552246417 and parameters: {'C': 39.51461734610825, 'gamma': 6.786163920573438}. Best is trial 66 with value: 0.5701049939691195.
[I 2024-11-28 23:34:38,327] Trial 94 finished with value: 0.5426764242277952 and parameters: {'C': 30.642698644976743, 'gamma': 6.941845172466869}. Best is trial 66 with value: 0.5701049939691195.
[I 2024-11-28 23:

In [11]:
# loading in hyperparameters
#bestparams_MG_svm = functions.extract_bestparams('MatchingGame', model_type = 'svm')


In [12]:
# training and testing MatchingGame model: OBS giving one prediction per participant

MG_training_svm = functions.modeltraining(data_MatchingGame,
             MG_features,
             bestparams_MG_svm,
             'svm',
             'MatchingGame',
             save = True)

In [13]:
# model testing

MG_MG_20_test_svm = functions.modeltesting2(
             MG_features,
             MG_MG_20,
             'svm',
             save = True,
             test_name = "MG_MG_20",
             train_name = 'MatchingGame')

MG_Qfam_80_test_svm = functions.modeltesting2(
             MG_features,
             MG_Qfam_80,
             'svm',
             save = True,
             test_name = "MG_Qfam_80",
             train_name = 'MatchingGame')

MG_Qfam_20_test_svm = functions.modeltesting2(
             MG_features,
             MG_Qfam_20,
             'svm',
             save = True,
             test_name = "MG_Qfam_20",
             train_name = 'MatchingGame')

### Questions Familiar - svm

In [14]:
# SVM PARAMETERTUNING
bestparams_Qfam_svm = functions.parametertuner(model_type = 'svm', train = data_Questions_fam, feature_lists = Qfam_features, train_name = 'Q_fam', trials = 100, jobs = 32)

[I 2024-11-28 23:47:41,779] A new study created in memory with name: no-name-2cc4acea-c294-463d-9d98-546bc97e23dd


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
2
3
3
3
3
3
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
3
3
4
3
4
3
3
3
3
3
3
4
3
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
5
4
5
5
5
5
5
5
5
5
5
5
5
5
5
5


[I 2024-11-28 23:48:31,259] Trial 9 finished with value: 0.5568483643361181 and parameters: {'C': 35.45641311812199, 'gamma': 1.7337250416296954}. Best is trial 9 with value: 0.5568483643361181.


1
5
5
5
5
5
5
5


[I 2024-11-28 23:48:32,657] Trial 28 finished with value: 0.4389259151862552 and parameters: {'C': 68.06994341152843, 'gamma': 76.94414857241885}. Best is trial 9 with value: 0.5568483643361181.


...

[I 2024-11-28 23:50:25,597] Trial 94 finished with value: 0.4468363382847368 and parameters: {'C': 36.36202021045228, 'gamma': 23.27257055103996}. Best is trial 46 with value: 0.5599156963606198.


5
5
5


[I 2024-11-28 23:50:28,916] Trial 96 finished with value: 0.450687876825003 and parameters: {'C': 0.5052067795764756, 'gamma': 21.38812036482328}. Best is trial 46 with value: 0.5599156963606198.
[I 2024-11-28 23:50:30,048] Trial 93 finished with value: 0.4426335971219978 and parameters: {'C': 18.217908377465005, 'gamma': 25.381048778425562}. Best is trial 46 with value: 0.5599156963606198.
[I 2024-11-28 23:50:30,124] Trial 97 finished with value: 0.44661981497534686 and parameters: {'C': 44.89967058555124, 'gamma': 23.316691796100816}. Best is trial 46 with value: 0.5599156963606198.


5


[I 2024-11-28 23:50:32,047] Trial 95 finished with value: 0.4484011532051079 and parameters: {'C': 46.14148566116515, 'gamma': 21.719376646598178}. Best is trial 46 with value: 0.5599156963606198.
[I 2024-11-28 23:50:32,519] Trial 99 finished with value: 0.4489360900141636 and parameters: {'C': 33.66892344736219, 'gamma': 21.412724768366797}. Best is trial 46 with value: 0.5599156963606198.
[I 2024-11-28 23:50:35,023] Trial 98 finished with value: 0.4840049787327665 and parameters: {'C': 45.01377672748936, 'gamma': 13.049159532098685}. Best is trial 46 with value: 0.5599156963606198.


In [15]:
Qfam_training_svm = functions.modeltraining2(data_Questions_fam,
             Qfam_features,
             bestparams_Qfam_svm,
             'svm',
             'Qfam',
             save = True)

In [16]:
Qfam_qfam_20_test_svm = functions.modeltesting2(
             Qfam_features,
             Qfam_Qfam_20,
             'svm',
             save = True,
             test_name = "Qfam_Qfam_20",
             train_name = 'Qfam')

Qfam_MG_80_test_svm = functions.modeltesting2(
             Qfam_features,
             Qfam_MG_80,
             'svm',
             save = True,
             test_name = "Qfam_MG_80",
             train_name = 'Qfam')

Qfam_MG_20_test_svm = functions.modeltesting2(
             Qfam_features,
             Qfam_MG_20,
             'svm',
             save = True,
             test_name = "Qfam_MG_20",
             train_name = 'Qfam')

Qfam_Qunfam_20_test_svm = functions.modeltesting2(
             Qfam_features,
             Qfam_Qunfam_20,
             'svm',
             save = True,
             test_name = "Qfam_Qunfam_20",
             train_name = 'Qfam')

Qfam_Qunfam_80_test_svm = functions.modeltesting2(
             Qfam_features,
             Qfam_Qunfam_80,
             'svm',
             save = True,
             test_name = "Qfam_Qunfam_80",
             train_name = 'Qfam')

/home/ucloud/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/ucloud/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/ucloud/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/home/ucloud/.local/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

### Questions familiar - xgb

In [17]:
# XBG PARAMETERTUNING
bestparams_Qfam_xgb = functions.parametertuner(model_type = 'xgb', train = data_Questions_fam, feature_lists = Qfam_features, train_name = 'Q_fam', trials = 100, jobs = 32)

[I 2024-11-28 23:53:52,054] A new study created in memory with name: no-name-7b45097e-5e66-4eef-873c-d7e90db54447


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
33

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
3
3
3
3
3
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
44

4
4
5
4
5
5
5
5
5
5
5
4
5
5
5
5
5
5
5
5
5
5
5


[I 2024-11-28 23:54:45,253] Trial 2 finished with value: 0.5368525191761662 and parameters: {'learning_rate': 0.24950234485658213, 'reg_lambda': 0.0025028383030512324, 'max_depth': 7, 'min_child_weight': 20, 'gamma': 4.1195331961815125, 'reg_alpha': 2.8711833753562454e-05}. Best is trial 2 with value: 0.5368525191761662.


5
1
5
5
5
5
5


[I 2024-11-28 23:54:47,179] Trial 4 finished with value: 0.5277058603217402 and parameters: {'learning_rate': 0.17999975429150603, 'reg_lambda': 1.5706411837825567, 'max_depth': 7, 'min_child_weight': 27, 'gamma': 4.86459653990004, 'reg_alpha': 6.357384695982359e-05}. Best is trial 2 with value: 0.5368525191761662.


...

[I 2024-11-28 23:57:27,870] Trial 94 finished with value: 0.5395733125657659 and parameters: {'learning_rate': 0.1419819769090054, 'reg_lambda': 0.0033767618213952372, 'max_depth': 8, 'min_child_weight': 25, 'gamma': 2.219595875252385, 'reg_alpha': 0.00022482489311637408}. Best is trial 67 with value: 0.5475463071218784.


5
5


[I 2024-11-28 23:57:28,831] Trial 97 finished with value: 0.5382163767105791 and parameters: {'learning_rate': 0.1356879411913185, 'reg_lambda': 0.002694300029833264, 'max_depth': 8, 'min_child_weight': 26, 'gamma': 3.689172372424786, 'reg_alpha': 0.0002642311258623086}. Best is trial 67 with value: 0.5475463071218784.


5


[I 2024-11-28 23:57:29,404] Trial 96 finished with value: 0.5371537581404419 and parameters: {'learning_rate': 0.13996383735302978, 'reg_lambda': 0.0021495965026014923, 'max_depth': 8, 'min_child_weight': 24, 'gamma': 2.292002548037185, 'reg_alpha': 0.0003352612577335185}. Best is trial 67 with value: 0.5475463071218784.
[I 2024-11-28 23:57:29,493] Trial 98 finished with value: 0.5357105947837756 and parameters: {'learning_rate': 0.14352989597981491, 'reg_lambda': 0.0020691472466373307, 'max_depth': 8, 'min_child_weight': 25, 'gamma': 2.2058131722052643, 'reg_alpha': 0.0002298240232510115}. Best is trial 67 with value: 0.5475463071218784.
[I 2024-11-28 23:57:29,829] Trial 99 finished with value: 0.5354329753800203 and parameters: {'learning_rate': 0.1436887063426116, 'reg_lambda': 0.001709373350715247, 'max_depth': 8, 'min_child_weight': 26, 'gamma': 2.3220679882528503, 'reg_alpha': 0.011505254020303616}. Best is trial 67 with value: 0.5475463071218784.


In [18]:
# training
Qfam_training_xgb = functions.modeltraining(data_Questions_fam,
             Qfam_features,
             bestparams_Qfam_xgb,
             'xgb',
             'Qfam',
             save = True)

In [19]:
# testing
Qfam_qfam_20_test_xgb = functions.modeltesting2(
             Qfam_features,
             Qfam_Qfam_20,
             'xgb',
             save = True,
             test_name = "Qfam_Qfam_20",
             train_name = 'Qfam')

Qfam_MG_80_test_xgb = functions.modeltesting2(
             Qfam_features,
             Qfam_MG_80,
             'xgb',
             save = True,
             test_name = "Qfam_MG_80",
             train_name = 'Qfam')

Qfam_MG_20_test_xgb = functions.modeltesting2(
             Qfam_features,
             Qfam_MG_20,
             'xgb',
             save = True,
             test_name = "Qfam_MG_20",
             train_name = 'Qfam')

Qfam_Qunfam_20_test_xgb = functions.modeltesting2(
             Qfam_features,
             Qfam_Qunfam_20,
             'xgb',
             save = True,
             test_name = "Qfam_Qunfam_20",
             train_name = 'Qfam')

Qfam_Qunfam_80_test_xgb = functions.modeltesting2(
             Qfam_features,
             Qfam_Qunfam_80,
             'xgb',
             save = True,
             test_name = "Qfam_Qunfam_80",
             train_name = 'Qfam')

### Questions Unfamiliar - XGB

In [20]:
# XGB PARAMETERTUNING
bestparams_Qunfam_xgb = functions.parametertuner(model_type = 'xgb', train = data_Questions_unfam, feature_lists = Qunfam_features, train_name = 'Q_unfam', trials = 100, jobs = 32)

[I 2024-11-28 23:57:35,050] A new study created in memory with name: no-name-04492029-8e35-4538-83fc-d9e863b55d35


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
22

2
2
2
2
2
2
2
2
22

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
5
4
5
4
4
4
5
4
4
5
5
5
55

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5


[I 2024-11-28 23:58:29,461] Trial 1 finished with value: 0.5658967624336496 and parameters: {'learning_rate': 0.27721723889521777, 'reg_lambda': 6.363893245129931e-06, 'max_depth': 8, 'min_child_weight': 80, 'gamma': 4.9656177777589345, 'reg_alpha': 0.025302587908493525}. Best is trial 1 with value: 0.5658967624336496.


1


[I 2024-11-28 23:58:29,793] Trial 3 finished with value: 0.5736424141508382 and parameters: {'learning_rate': 0.21899778811783155, 'reg_lambda': 0.05598388973791546, 'max_depth': 4, 'min_child_weight': 90, 'gamma': 4.194389840021003, 'reg_alpha': 0.0010783691742323454}. Best is trial 3 with value: 0.5736424141508382.


...

[I 2024-11-29 00:00:56,132] Trial 92 finished with value: 0.5896954946787668 and parameters: {'learning_rate': 0.013657401673506854, 'reg_lambda': 3.185580418337128e-08, 'max_depth': 4, 'min_child_weight': 44, 'gamma': 2.8731802163488718, 'reg_alpha': 19.281056293008866}. Best is trial 43 with value: 0.6003022091026592.
[I 2024-11-29 00:00:56,583] Trial 93 finished with value: 0.5843346048859165 and parameters: {'learning_rate': 0.0187571976339434, 'reg_lambda': 2.796291665524092e-08, 'max_depth': 4, 'min_child_weight': 45, 'gamma': 3.9047988967282454, 'reg_alpha': 17.45497222388049}. Best is trial 43 with value: 0.6003022091026592.
[I 2024-11-29 00:00:56,640] Trial 89 finished with value: 0.5877884250231897 and parameters: {'learning_rate': 0.02186817405267322, 'reg_lambda': 2.114338844553229e-08, 'max_depth': 7, 'min_child_weight': 45, 'gamma': 3.958943476464215, 'reg_alpha': 17.388915492535155}. Best is trial 43 with value: 0.6003022091026592.


2


[I 2024-11-29 00:00:56,914] Trial 94 finished with value: 0.5896982960070518 and parameters: {'learning_rate': 0.010817456736530633, 'reg_lambda': 3.4490404927326114e-07, 'max_depth': 4, 'min_child_weight': 44, 'gamma': 2.845064616143291, 'reg_alpha': 0.7540444483664868}. Best is trial 43 with value: 0.6003022091026592.
[I 2024-11-29 00:00:56,923] Trial 87 finished with value: 0.5786199287309459 and parameters: {'learning_rate': 0.038494984696036114, 'reg_lambda': 3.734644124731919e-08, 'max_depth': 10, 'min_child_weight': 55, 'gamma': 4.0611042578873695, 'reg_alpha': 11.10846078936261}. Best is trial 43 with value: 0.6003022091026592.
[I 2024-11-29 00:00:57,180] Trial 88 finished with value: 0.5907138065906599 and parameters: {'learning_rate': 0.01619678909928338, 'reg_lambda': 3.367288285275841e-08, 'max_depth': 7, 'min_child_weight': 56, 'gamma': 3.9671885285008335, 'reg_alpha': 8.803831467937686}. Best is trial 43 with value: 0.6003022091026592.
[I 2024-11-29 00:00:57,375] Trial 95

3
3
3
3
4
4
4
4
5
5
5
5


[I 2024-11-29 00:01:01,696] Trial 96 finished with value: 0.5793415639059509 and parameters: {'learning_rate': 0.03546117046041376, 'reg_lambda': 3.387722715235004e-08, 'max_depth': 4, 'min_child_weight': 44, 'gamma': 2.88652380508637, 'reg_alpha': 19.034046646885827}. Best is trial 43 with value: 0.6003022091026592.
[I 2024-11-29 00:01:01,797] Trial 97 finished with value: 0.5838345574896564 and parameters: {'learning_rate': 0.0266236270315639, 'reg_lambda': 1.1948410467316698e-05, 'max_depth': 4, 'min_child_weight': 56, 'gamma': 2.979851847927751, 'reg_alpha': 16.69728513359419}. Best is trial 43 with value: 0.6003022091026592.
[I 2024-11-29 00:01:01,882] Trial 98 finished with value: 0.5842074465519808 and parameters: {'learning_rate': 0.03198624431011174, 'reg_lambda': 4.397316661102563e-07, 'max_depth': 4, 'min_child_weight': 45, 'gamma': 3.0543210471826017, 'reg_alpha': 24.400002271570536}. Best is trial 43 with value: 0.6003022091026592.
[I 2024-11-29 00:01:01,958] Trial 99 fini

In [21]:
Qfam_training_xgb = functions.modeltraining(data_Questions_unfam,
             Qunfam_features,
             bestparams_Qunfam_xgb,
             'xgb',
             'Qunfam',
             save = True)

In [8]:
Qunfam_qunfam_20_test_xgb = functions.modeltesting2(
             Qunfam_features,
             Qunfam_Qunfam_20,
             'xgb',
             save = True,
             test_name = "Qunfam_Qunfam_20",
             train_name = 'Qunfam')

Qunfam_Qfam_80_test_xgb = functions.modeltesting2(
             Qunfam_features,
             Qunfam_Qfam_80,
             'xgb',
             save = True,
             test_name = "Qunfam_Qfam_80",
             train_name = 'Qunfam')

Qunfam_Q_fam_20_test_xgb = functions.modeltesting2(
             Qunfam_features,
             Qunfam_Qfam_20,
             'xgb',
             save = True,
             test_name = "Qunfam_Qfam_20",
             train_name = 'Qunfam')

## Q unfam - svm

In [ ]:
# SVM PARAMETERTUNING
bestparams_Qunfam_svm = functions.parametertuner(model_type = 'svm', train = data_Questions_unfam, feature_lists = Qunfam_features, train_name = 'Q_unfam', trials = 100, jobs = 32)

[I 2024-11-29 08:43:06,100] A new study created in memory with name: no-name-9cd45776-28aa-47b6-bc3b-b164dcab3d3d


1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
11

2
2
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
4
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
4
5
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


[I 2024-11-29 08:44:05,996] Trial 24 finished with value: 0.56981810572082 and parameters: {'C': 99.32201207660607, 'gamma': 0.3009044000247068}. Best is trial 24 with value: 0.56981810572082.


5
1
4
4
4
4
5
4
4
4
5
5
5
5
5


In [ ]:

Qunfam_training_svm = functions.modeltraining(data_Questions_unfam,
             Qunfam_features,
             bestparams_Qunfam_svm,
             'svm',
             'Qunfam',
             save = True)

In [ ]:
Qunfam_Qunfam_20_test_svm = functions.modeltesting2(
             Qunfam_features,
             Qunfam_Qunfam_20,
             'svm',
             save = True,
             test_name = "Qunfam_Qunfam_20",
             train_name = 'Qunfam')

In [ ]:
Qunfam_Qfam_20_test_svm = functions.modeltesting2(
             Qunfam_features,
             Qunfam_Qfam_20,
             'svm',
             save = True,
             test_name = "Qunfam_Qfam_20",
             train_name = 'Qunfam')

Qunfam_Qfam_80_test_svm = functions.modeltesting2(
             Qunfam_features,
             Qunfam_Qfam_80,
             'svm',
             save = True,
             test_name = "Qunfam_Qfam_80",
             train_name = 'Qunfam')

NameError: name 'Qunfam_features' is not defined

# extracting model performance across test set and visualizing them

In [ ]:
# MatchingGame
plot_values = functions.plot_estimates('MatchingGame', ['MG_MG_20', 'MG_Qfam_80', 'MG_Qfam_20'])
functions.plot_results('MatchingGame', plot_values)


In [ ]:
# Qfam
plot_values = functions.plot_estimates('Qfam', ['Qfam_Qfam_20', 'Qfam_MG_80', 'Qfam_MG_20', 'Qfam_Qunfam_80', 'Qfam_Qunfam_20'])
functions.plot_results('Qfam', plot_values)

In [ ]:
# Qunfam
plot_values = functions.plot_estimates('Qunfam', ['Qunfam_Qunfam_20', 'Qunfam_Qfam_80', 'Qunfam_Qfam_20'])
functions.plot_results('Qunfam', plot_values)